# 1. Data Acquisition
1.1 import Basic Libraries

1.2 Load the datasets

In [1]:
pip install spacy

In [2]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import Libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import string
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import Word
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


import warnings
warnings.filterwarnings("ignore")

In [4]:
# from google.colab import files
# upload = files.upload()

In [5]:
reviews=pd.read_excel('hotel_reviews (2).xlsx')

# 2. Exploratory Data Analysis (EDA)
2.1 Data Exploration

2.2 Checking duplicates & null values

2.3 Removing Unwanted features

After examining "@", we came to the conclusion that this columns is not useful in our Modelling. We will drop those column.
2.4 Checking the data with different aspects before preprocessing it.

In [6]:
reviews = pd.read_excel('hotel_reviews (2).xlsx')
reviews.drop(columns={'@'},inplace=True)
print("the shape of this dataset is :",reviews.shape)
reviews.sample(5)

the shape of this dataset is : (20491, 2)


,Review,Rating
848,good service not aware hotel haunted till chec...,4
7891,fantastic weekend birthday year husband surpri...,5
10293,tiggertastic like people read poor reviews hot...,4
4395,nice stayed hotel nov 19-23. great little bout...,4
11011,friendly florence warmly welcomed friendly hel...,5


# 3. Text-PreProcessing
3.1 Cleaning data

There are a number of special characters and other values that needs to be removed. We will define a clean text function to remove it.
This function will convert all text to lower case
3.2 Removing Emojis

certain useless emojis will be added to data which should be removed custom emojis list.
3.3 Chat words abbrevation replacements

Short forms like IDK, WTF, FYI etc will be replaced with their abbrevations.
3.4 Tokenization

To breaks text into smaller parts for easier to remove stopwords, doing Stemming & Lemmatization.
3.6 Spell Checker

to check every spelling and correct it if it is wrong
3.7 StopWords

to eliminate words that are so widely used that they carry very little useful information.
3.8 Stemming

reducing a word to its word stem that suffixes or the roots
3.9 Lemmatization

lemmatization is to reduce a word to its root form

In [7]:
# Data Pre-Processing
df = reviews[["Review"]]
df.head()

,Review
0,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...
3,"unique, great stay, wonderful time hotel monac..."
4,"great stay great stay, went seahawk game aweso..."


In [8]:
# convert to lower case and removing punctuations,lowercase, remove text in square brackets,remove links,remove punctuation and remove words containing numbers.
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    #text = re.sub('[^a-z]', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://*', '', text)
    text = re.sub("\www.*", "", text)
    text = re.sub("\.com*", "", text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [9]:
df["Clean_Reviews"] = df["Review"].apply(lambda x: clean_text(x))
df.head()

,Review,Clean_Reviews
0,nice hotel expensive parking got good deal sta...,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,nice rooms not experience hotel monaco seattl...
3,"unique, great stay, wonderful time hotel monac...",unique great stay wonderful time hotel monaco ...
4,"great stay great stay, went seahawk game aweso...",great stay great stay went seahawk game awesom...


In [10]:
lower_words = [text.lower() for text in df.Clean_Reviews]
print(lower_words[0:50])

['nice hotel expensive parking got good deal stay hotel anniversary arrived late evening took advice previous reviews did valet parking check quick easy little disappointed nonexistent view room room clean nice size bed comfortable woke stiff neck high pillows not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway maybe just noisy neighbors aveda bath products nice did not goldfish stay nice touch taken advantage staying longer location great walking distance shopping overall nice experience having pay  parking night  ', 'ok nothing special charge diamond member hilton decided chain shot  anniversary seattle start booked suite paid extra website description not suite bedroom bathroom standard hotel room took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast got kidding embassy suits sitting room bathroom bedroom unlike kimpton calls suite

In [11]:
# # Removing emojis
# def remove_emoji(text):
#     emoji_pattern = re.compile("["
#                            u"\U0001F600-\U0001F64F"  # emoticons
#                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            u"\U00002702-\U000027B0"
#                            u"\U000024C2-\U0001F251"
#                            "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', text)

In [12]:
# df["Clean_Reviews"] = df["Clean_Reviews"].apply(lambda x: remove_emoji(x))
# df.head()

In [13]:
# Handling Chat word
'''
IDK = I Dont Know
ASAP = as soon as possible
FYI = For Your Information
FIMH = Forever in my heart
'''
chat_words = {  "AFAIK": "As Far As I Know", "AFK": "Away From Keyboard",
                "ASAP": "As Soon As Possible", "ATK":"At The Keyboard",
                "ATM": "At The Moment", "A3": "Anytime, Anywhere, Anyplace",
                "BAK": "Back At Keyboard" ,"BBL": "Be Back Later",
                "BBS":"Be Back Soon", "BFN":"Bye For Now",
                "B4N":"Bye For Now", "BRB":"Be Right Back",
                "BRT":"Be Right There", "BTW":"By The Way",
                "B4":"Before", "B4N":"Bye For Now",
                "CU":"See You", "CUL8R":"See You Later",
                "CYA":"See You", "FAQ":"Frequently Asked Questions",
                "FC":"Fingers Crossed", "FWIW":"For What It's Worth",
                "FYI":"For Your Information", "GAL":"Get A Life",
                "GG":"Good Game", "GN":"Good Night",
                "GMTA":"Great Minds Think Alike", "GR8":"Great",
                "G9":"Genius", "IC":"I See",
                "ICQ":"I Seek you", "ILU": "I Love You",
                "IMHO":"In My Honest Humble Opinion", "IMO":"In My Opinion",
                "IOW":"In Other Words" ,"IRL":"In Real Life",
                "KISS":"Keep It Simple Stupid", "LDR":"Long Distance Relationship",
                "LMAO":"Laugh My Ass Off", "LOL":"Laughing Out Loud",
                "LTNS":"Long Time No See", "L8R":"Later",
                "MTE":"My Thoughts Exactly", "M8":"Mate",
                "NRN":"No Reply Necessary",  "OIC":"Oh I See",
                "PITA":"Pain In The Ass", "PRT":"Party",
                "PRW":"Parents Are Watching", "ROFL":"Rolling On The Floor Laughing",
                "ROFLOL":"Rolling On The Floor Laughing Out Loud", "ROTFLMAO":"Rolling On The Floor Laughing My Ass Off",
                "SK8":"Skate", 'STATS':'Your sex and age',
                "ASL":"Age Sex Location",  "THNX":"Thank You",
                "TTFN":"TaTa For Now", "TTYL":"Talk To You Later",
                "U":"You", "U2":"You Too",
                "U4E":"Yours For Ever", "WB":"Welcome Back",
                "WTF":"What The Fuck", "WTG":"Way To Go",
                "WUF":"Where Are You From",  "W8":"Wait",
                "7K":"Sick Laughter",  "TFW" : "That feeling when",
                "MFW" : "My face when", "MRW" : "My reaction when",
                "IFYP" : "I feel your pain", "LOL" : "Laughing out loud",
                "TNTL" : "Trying not to laugh", "JK" : "Just kidding",
                "IDC" : "I dont care", "ILY" : "I love you",
                "IMU" : "I miss you", "ADIH" : "Another day in hell",
                "IDC" : "I dont care" ,"ZZZ" : "Sleeping bored tired",
                "WYWH" : "Wish you were here", "TIME" : "Tears in my eyes",
                "BAE" : "Before anyone else", "FIMH" : "Forever in my heart",
                "BSAAW" : "Big smile and a wink", "BWL" : "Bursting with laughter",
                "LMAO" : "Laughing my ass off", "BFF": "Best friends forever",
                "CSL" : "Cant stop laughing"}


chat_words_dict = pd.DataFrame.from_dict(chat_words, orient="index",columns=['full_form'])
chat_words_dict['short_form'] = chat_words_dict.index
chat_words = chat_words_dict.reset_index()
chat_words.drop(["index"], inplace=True, axis=1)

In [14]:
chat_words.head()

,full_form,short_form
0,As Far As I Know,AFAIK
1,Away From Keyboard,AFK
2,As Soon As Possible,ASAP
3,At The Keyboard,ATK
4,At The Moment,ATM


In [15]:
def chat_conversion(text):
  new_text=[]
  for words in text.split():
    if words.upper() in chat_words['short_form']:
      new_text.append(chat_words[words["full_form"]])
    else:
      new_text.append(words)
  return " ".join(new_text)

In [16]:
    df["Clean_Reviews"] = df["Clean_Reviews"].apply(lambda x: chat_conversion(x))
    df.head()

,Review,Clean_Reviews
0,nice hotel expensive parking got good deal sta...,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,nice rooms not experience hotel monaco seattle...
3,"unique, great stay, wonderful time hotel monac...",unique great stay wonderful time hotel monaco ...
4,"great stay great stay, went seahawk game aweso...",great stay great stay went seahawk game awesom...


In [17]:
review = [text.strip() for text in df.Clean_Reviews] # remove both the leading and the trailing characters
review = [text for text in review if text] # removes empty strings, because they are considered in Python as False
review[:2]

['nice hotel expensive parking got good deal stay hotel anniversary arrived late evening took advice previous reviews did valet parking check quick easy little disappointed nonexistent view room room clean nice size bed comfortable woke stiff neck high pillows not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway maybe just noisy neighbors aveda bath products nice did not goldfish stay nice touch taken advantage staying longer location great walking distance shopping overall nice experience having pay parking night',
 'ok nothing special charge diamond member hilton decided chain shot anniversary seattle start booked suite paid extra website description not suite bedroom bathroom standard hotel room took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast got kidding embassy suits sitting room bathroom bedroom unlike kimpton calls suite da

In [18]:
# Joining the list into one string/text
review_text = ' '.join(review)
print(len(review_text))
print(review_text[0:150])

14220702
nice hotel expensive parking got good deal stay hotel anniversary arrived late evening took advice previous reviews did valet parking check quick easy


In [19]:
# Tokenization
nltk.download('punkt')
import nltk

from nltk.tokenize import word_tokenize
review_tokens = word_tokenize(review_text)
print(review_tokens[0:50])

# for Clean_Reviews Column
df['Clean_Reviews'] = df['Clean_Reviews'].apply(lambda x: nltk.word_tokenize(x))
df.head(5)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['nice', 'hotel', 'expensive', 'parking', 'got', 'good', 'deal', 'stay', 'hotel', 'anniversary', 'arrived', 'late', 'evening', 'took', 'advice', 'previous', 'reviews', 'did', 'valet', 'parking', 'check', 'quick', 'easy', 'little', 'disappointed', 'nonexistent', 'view', 'room', 'room', 'clean', 'nice', 'size', 'bed', 'comfortable', 'woke', 'stiff', 'neck', 'high', 'pillows', 'not', 'soundproof', 'like', 'heard', 'music', 'room', 'night', 'morning', 'loud', 'bangs', 'doors']


,Review,Clean_Reviews
0,nice hotel expensive parking got good deal sta...,"[nice, hotel, expensive, parking, got, good, d..."
1,ok nothing special charge diamond member hilto...,"[ok, nothing, special, charge, diamond, member..."
2,nice rooms not 4* experience hotel monaco seat...,"[nice, rooms, not, experience, hotel, monaco, ..."
3,"unique, great stay, wonderful time hotel monac...","[unique, great, stay, wonderful, time, hotel, ..."
4,"great stay great stay, went seahawk game aweso...","[great, stay, great, stay, went, seahawk, game..."


In [20]:
nltk.download('stopwords')

from nltk.corpus import stopwords

# # For wordCloud
# my_stop_words = stopwords.words('english')
# sw_list = ['\x92','rt','ye','yeah','haha','Yes','__Ç_éÈe','I','__Ç','__Ç_é_',"br/", "hotel", "room"]     # index 1044 emojis
# my_stop_words.extend(sw_list)

# no_stop_tokens = [word for word in review_tokens if not word in my_stop_words]

# For Clean_Reviews Column
my_stop_words=stopwords.words('english')
my_stop_words.remove('not')
sw_list = ['\x92','rt','ye','yeah','haha','Yes','__Ç_éÈe','I','__Ç','__Ç_é_',"br/","hotel","room"]     # index 1044 emojis
my_stop_words.extend(sw_list)
stopwords_set = set(my_stop_words)
cleaned_tweets = []

for index, row in df.iterrows():

    # filerting out all the stopwords
    words_without_stopwords = [word for word in row.Clean_Reviews if not word in stopwords_set and '#' not in word.lower()]

    # finally creating tweets list of tuples containing stopwords(list) and sentimentType
    cleaned_tweets.append(words_without_stopwords)

df['Clean_Reviews'] = cleaned_tweets
df.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review,Clean_Reviews
0,nice hotel expensive parking got good deal sta...,"[nice, expensive, parking, got, good, deal, st..."
1,ok nothing special charge diamond member hilto...,"[ok, nothing, special, charge, diamond, member..."
2,nice rooms not 4* experience hotel monaco seat...,"[nice, rooms, not, experience, monaco, seattle..."
3,"unique, great stay, wonderful time hotel monac...","[unique, great, stay, wonderful, time, monaco,..."
4,"great stay great stay, went seahawk game aweso...","[great, stay, great, stay, went, seahawk, game..."
5,love monaco staff husband stayed hotel crazy w...,"[love, monaco, staff, husband, stayed, crazy, ..."
6,"cozy stay rainy city, husband spent 7 nights m...","[cozy, stay, rainy, city, husband, spent, nigh..."
7,"excellent staff, housekeeping quality hotel ch...","[excellent, staff, housekeeping, quality, choc..."
8,"hotel stayed hotel monaco cruise, rooms genero...","[stayed, monaco, cruise, rooms, generous, deco..."
9,excellent stayed hotel monaco past w/e delight...,"[excellent, stayed, monaco, past, delight, rec..."


In [21]:
# Stemming
from nltk.stem import PorterStemmer
ps=PorterStemmer()

# stemmed_tokens=[ps.stem(word) for word in df.Clean_Reviews]
# creating other Column
df['stemmed'] = df['Clean_Reviews'].apply(lambda x: " ".join([ps.stem(word) for word in x]))
df.head()

,Review,Clean_Reviews,stemmed
0,nice hotel expensive parking got good deal sta...,"[nice, expensive, parking, got, good, deal, st...",nice expens park got good deal stay anniversar...
1,ok nothing special charge diamond member hilto...,"[ok, nothing, special, charge, diamond, member...",ok noth special charg diamond member hilton de...
2,nice rooms not 4* experience hotel monaco seat...,"[nice, rooms, not, experience, monaco, seattle...",nice room not experi monaco seattl good nt lev...
3,"unique, great stay, wonderful time hotel monac...","[unique, great, stay, wonderful, time, monaco,...",uniqu great stay wonder time monaco locat exce...
4,"great stay great stay, went seahawk game aweso...","[great, stay, great, stay, went, seahawk, game...",great stay great stay went seahawk game awesom...


In [22]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [23]:
# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# Create an instance of WordNetLemmatizer
wnl = WordNetLemmatizer()

lemma_words=[wnl.lemmatize(word) for word in lower_words]
print(lemma_words[:50])

df['lemmas'] = df['Clean_Reviews'].apply(lambda x: " ".join([wnl.lemmatize(word) for word in x]))
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['nice hotel expensive parking got good deal stay hotel anniversary arrived late evening took advice previous reviews did valet parking check quick easy little disappointed nonexistent view room room clean nice size bed comfortable woke stiff neck high pillows not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway maybe just noisy neighbors aveda bath products nice did not goldfish stay nice touch taken advantage staying longer location great walking distance shopping overall nice experience having pay  parking night  ', 'ok nothing special charge diamond member hilton decided chain shot  anniversary seattle start booked suite paid extra website description not suite bedroom bathroom standard hotel room took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast got kidding embassy suits sitting room bathroom bedroom unlike kimpton calls suite

,Review,Clean_Reviews,stemmed,lemmas
0,nice hotel expensive parking got good deal sta...,"[nice, expensive, parking, got, good, deal, st...",nice expens park got good deal stay anniversar...,nice expensive parking got good deal stay anni...
1,ok nothing special charge diamond member hilto...,"[ok, nothing, special, charge, diamond, member...",ok noth special charg diamond member hilton de...,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,"[nice, rooms, not, experience, monaco, seattle...",nice room not experi monaco seattl good nt lev...,nice room not experience monaco seattle good n...
3,"unique, great stay, wonderful time hotel monac...","[unique, great, stay, wonderful, time, monaco,...",uniqu great stay wonder time monaco locat exce...,unique great stay wonderful time monaco locati...
4,"great stay great stay, went seahawk game aweso...","[great, stay, great, stay, went, seahawk, game...",great stay great stay went seahawk game awesom...,great stay great stay went seahawk game awesom...


In [24]:
clean_reviews=' '.join(lemma_words)
clean_reviews[:500]

'nice hotel expensive parking got good deal stay hotel anniversary arrived late evening took advice previous reviews did valet parking check quick easy little disappointed nonexistent view room room clean nice size bed comfortable woke stiff neck high pillows not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway maybe just noisy neighbors aveda bath products nice did not goldfish stay nice touch taken advantage staying longer location grea'

In [25]:
# from wordcloud import WordCloud

# # Define a function to plot word cloud
# def plot_cloud(wordcloud):
#     plt.figure(figsize=(40,30))
#     plt.imshow(wordcloud)
#     plt.axis('off')


# # Generate Word Cloud
# wordcloud=WordCloud(width=3000,height=2000,background_color='black',max_words=500,
#                      colormap='Set1',stopwords=my_stop_words).generate(clean_reviews)
# plot_cloud(wordcloud)

In [26]:
df.head()

,Review,Clean_Reviews,stemmed,lemmas
0,nice hotel expensive parking got good deal sta...,"[nice, expensive, parking, got, good, deal, st...",nice expens park got good deal stay anniversar...,nice expensive parking got good deal stay anni...
1,ok nothing special charge diamond member hilto...,"[ok, nothing, special, charge, diamond, member...",ok noth special charg diamond member hilton de...,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,"[nice, rooms, not, experience, monaco, seattle...",nice room not experi monaco seattl good nt lev...,nice room not experience monaco seattle good n...
3,"unique, great stay, wonderful time hotel monac...","[unique, great, stay, wonderful, time, monaco,...",uniqu great stay wonder time monaco locat exce...,unique great stay wonderful time monaco locati...
4,"great stay great stay, went seahawk game aweso...","[great, stay, great, stay, went, seahawk, game...",great stay great stay went seahawk game awesom...,great stay great stay went seahawk game awesom...


# Using--TF-IDF

In [27]:
dt = pd.DataFrame({"lemmas":df.lemmas, "rating" :reviews.Rating})
dt.head()

,lemmas,rating
0,nice expensive parking got good deal stay anni...,4
1,ok nothing special charge diamond member hilto...,2
2,nice room not experience monaco seattle good n...,3
3,unique great stay wonderful time monaco locati...,5
4,great stay great stay went seahawk game awesom...,5


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vectorizer=TfidfVectorizer(max_features=1000,max_df=0.5,smooth_idf=True)
doc_vec=vectorizer.fit_transform(dt.lemmas)
names_features=vectorizer.get_feature_names_out()
dense=doc_vec.todense()
denselist=dense.tolist()
df_=pd.DataFrame(denselist,columns=names_features)

In [30]:
df_

,able,absolutely,ac,access,accommodation,activity,actually,add,added,additional,...,worry,worse,worst,worth,write,wrong,year,yes,york,young
0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
2,0.0,0.00000,0.093496,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
4,0.0,0.00000,0.000000,0.0,0.0,0.083364,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20486,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
20487,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
20488,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
20489,0.0,0.06336,0.000000,0.0,0.0,0.000000,0.032006,0.042841,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.080047,0.000000,0.0,0.038648


In [31]:
text_to_check = dt.lemmas[209]
text_vectorized = vectorizer.transform([text_to_check])
dense_text = text_vectorized.todense()
df_text = pd.DataFrame(dense_text, columns=names_features)
df_text

,able,absolutely,ac,access,accommodation,activity,actually,add,added,additional,...,worry,worse,worst,worth,write,wrong,year,yes,york,young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
x2=doc_vec

In [33]:
dt['sentiment'] = pd.cut(dt['rating'], bins=[0, 2, 3, 5], labels=['negative', 'neutral', 'positive'])
dt.head()
y2 = dt["sentiment"]

## <span style='color:purple'>  Logistic Regression <span>

In [34]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x2,y2,test_size=0.2,random_state=42)

In [35]:
x_train.shape

(16392, 1000)

In [36]:
from sklearn.linear_model import LogisticRegression

model_lr=LogisticRegression()
model_lr.fit(x_train,y_train)

LogisticRegression()

In [37]:
print('accuracy using TF-IDF is :',model_lr.score(x_test,y_test))

accuracy using TF-IDF is : 0.8499634057087094


In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
pred=model_lr.predict(x_test)
lr_tfidf_report=classification_report(y_test,pred)
print(lr_tfidf_report)

              precision    recall  f1-score   support

    negative       0.76      0.74      0.75       625
     neutral       0.46      0.18      0.26       432
    positive       0.89      0.97      0.92      3042

    accuracy                           0.85      4099
   macro avg       0.70      0.63      0.65      4099
weighted avg       0.82      0.85      0.83      4099



#### <span style='color:red'> Interpretation of LR_TF-IDF:<span>

- By looking at support we can see that the data is imbalanced and the f1-score is the HM of precision and recall (gives more weight to the lower class). Since the value of f1-score(0.30) is not higher(<1) which indicates models performance was relatively low.

In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[ 462,   42,  121],
       [  94,   79,  259],
       [  48,   51, 2943]], dtype=int64)

In [40]:
sent = ["this hotel is good!"]
transformed = vectorizer.transform(sent)
prediction = model_lr.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['positive']


In [41]:
sent = ["this hotel is bad!"]
transformed = vectorizer.transform(sent)
prediction = model_lr.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['negative']


In [42]:
sent = ["this hotel was good but food where smelly"]
transformed = vectorizer.transform(sent)
prediction = model_lr.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['positive']


In [43]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [44]:
from imblearn.over_sampling import RandomOverSampler

In [45]:
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
x_train_resampled, y_train_resampled = oversampler.fit_resample(x_train, y_train)

In [46]:
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(x_train_resampled, y_train_resampled)
y_pred = logreg_model.predict(x_test)
lros_tfidf_report=classification_report(y_test,pred)
print(lros_tfidf_report)

              precision    recall  f1-score   support

    negative       0.76      0.74      0.75       625
     neutral       0.46      0.18      0.26       432
    positive       0.89      0.97      0.92      3042

    accuracy                           0.85      4099
   macro avg       0.70      0.63      0.65      4099
weighted avg       0.82      0.85      0.83      4099



In [47]:
sent = ["this hotel was good but bedsheets where smelly and having strains on them"]
transformed = vectorizer.transform(sent)
prediction = logreg_model.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['positive']


In [48]:
sent = ["the hotel was not good"]
transformed = vectorizer.transform(sent)
prediction = logreg_model.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['positive']


## <span style='color:purple'> Naive Bayes <span>

In [49]:
x_train,x_test,y_train,y_test=train_test_split(x2,y2,test_size=0.2,random_state=42)

In [50]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(x_train,y_train)

MultinomialNB()

In [51]:
y_pred=mnb.predict(x_test)

In [52]:
nb_tfidf_report=classification_report(y_test,y_pred)
print(nb_tfidf_report)

              precision    recall  f1-score   support

    negative       0.86      0.48      0.61       625
     neutral       0.00      0.00      0.00       432
    positive       0.81      1.00      0.89      3042

    accuracy                           0.81      4099
   macro avg       0.56      0.49      0.50      4099
weighted avg       0.73      0.81      0.76      4099



In [53]:
sent = ["this hotel was good but food where smelly"]
transformed = vectorizer.transform(sent)
prediction = mnb.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['positive']


In [54]:
from imblearn.over_sampling import RandomOverSampler

oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
x_train_resampled, y_train_resampled = oversampler.fit_resample(x_train, y_train)

In [55]:
from sklearn.naive_bayes import MultinomialNB
mnb_os=MultinomialNB()
mnb_os.fit(x_train_resampled,y_train_resampled)
y_pred = mnb_os.predict(x_test)
nbos_tfidf_report=classification_report(y_test,y_pred)
print(nbos_tfidf_report)

              precision    recall  f1-score   support

    negative       0.61      0.76      0.68       625
     neutral       0.29      0.58      0.38       432
    positive       0.96      0.77      0.86      3042

    accuracy                           0.75      4099
   macro avg       0.62      0.70      0.64      4099
weighted avg       0.84      0.75      0.78      4099



In [56]:
sent = ["this hotel was good but food where smelly"]
transformed = vectorizer.transform(sent)
prediction = mnb_os.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['neutral']


In [57]:
sent = ["the hotel was terrible, food were horrible, toilets were not cleaned"]
transformed = vectorizer.transform(sent)
prediction = mnb_os.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['negative']


In [58]:
sent = ["the hotel is bad"]
transformed = vectorizer.transform(sent)
prediction = mnb_os.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['negative']


In [59]:
sent = ["the hotel was good"]
transformed = vectorizer.transform(sent)
prediction = mnb_os.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['neutral']


In [60]:
sent = ["the hotel was not good"]
transformed = vectorizer.transform(sent)
prediction = mnb_os.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['neutral']


In [61]:
sent = ["the hotel was awesome"]
transformed = vectorizer.transform(sent)
prediction = mnb_os.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['positive']


In [62]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# import joblib

# # Assuming you have your data and target variables
# x_train, x_test, y_train, y_test = train_test_split(x_tfidf, y_tfidf, test_size=0.2, random_state=42)

# # TF-IDF Vectorization
# vectorizer = TfidfVectorizer(max_features=1000, max_df=0.5, smooth_idf=True)
# x_train_tfidf = vectorizer.fit_transform(x_train)
# x_test_tfidf = vectorizer.transform(x_test)

# # Naive Bayes model
# mnb = MultinomialNB()
# mnb.fit(x_train_tfidf, y_train)

# # Save the TF-IDF Vectorizer and Multinomial Naive Bayes model using joblib
# joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
# joblib.dump(mnb, 'multinomial_nb_model.joblib')

# # Load the models later if needed
# loaded_vectorizer = joblib.load('tfidf_vectorizer.joblib')
# loaded_mnb = joblib.load('multinomial_nb_model.joblib')

# # Now you can use the loaded models for predictions
# y_pred = loaded_mnb.predict(loaded_vectorizer.transform(x_test))


In [63]:

# import joblib

# # Split the data
# x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

# # Resample the training data
# oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
# x_train_resampled, y_train_resampled = oversampler.fit_resample(x_train, y_train)

# # Train the Naive Bayes model
# mnb_os = MultinomialNB()
# mnb_os.fit(x_train_resampled, y_train_resampled)

# # Save the model to a file using joblib
# joblib.dump(mnb_os, 'naive_bayes_model.joblib')

# # Save the split data
# data_dict = {
#     'x_train': x_train,
#     'x_test': x_test,
#     'y_train': y_train,
#     'y_test': y_test,
#     'x_train_resampled': x_train_resampled,
#     'y_train_resampled': y_train_resampled
# }


# joblib.dump(data_dict, 'split_data.joblib')

# # Evaluate the model
# # y_pred = mnb_os.predict(x_test)
# # nbos_tfidf_report = classification_report(y_test, y_pred)
# # print(nbos_tfidf_report)


In [64]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [65]:
import joblib

# Save the TF-IDF Vectorizer
vectorizer_filename = 'tfidf_vectorizer.joblib'
joblib.dump(vectorizer, vectorizer_filename)

['tfidf_vectorizer.joblib']

In [66]:
# Save the Naive Bayes model
model_filename = 'naive_bayes_model.joblib'
joblib.dump(mnb_os, model_filename)

['naive_bayes_model.joblib']

In [67]:
# Create and fit the resampling object
ros = RandomOverSampler()
x_train_resampled, y_train_resampled = ros.fit_resample(x_train, y_train)

# Save the resampling object
resampling_filename = 'random_oversampler.joblib'
joblib.dump(ros, resampling_filename)

['random_oversampler.joblib']